In [16]:
!conda create -n crewai python=3.11 --y
!conda activate crewai



/bin/bash: line 1: conda: command not found
/bin/bash: line 1: conda: command not found


In [17]:
%pip install -U 'crewai[tools]' mdpdf

In [24]:
# export OPENAI_API_KEY=xxxxxxxxxx
!export GROQ_API_KEY=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

In [25]:
from crewai import Agent, Task, Crew, Process
from crewai_tools import tool
# from langchain_groq import ChatOpenAI
from crewai_tools.tools import FileReadTool
import os, requests, re, mdpdf, subprocess
from openai import OpenAI
from langchain_groq import ChatGroq

In [26]:
!pip install langchain_groq

In [30]:
from google.colab import userdata
userdata.get('GROQ_API_KEY')

'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'

In [34]:
os.environ["GROQ_API_KEY"] = "gsk_IVLryi9NbnbWgD0ZsjzqWGdyb3FYIevpAIRz4gOp5MhE6j0ddHoApi_key"

In [35]:
llm =ChatGroq(
    groq_api_base="https://api.groq.com/openai/v1", # https://api.openai.com/v1 or https://api.groq.com/openai/v1
    groq_api_key=os.getenv("GROQ_API_KEY"), # os.getenv("OPENAI_API_KEY") or os.getenv("GROQ_API_KEY")
    model_name="mixtral-8x7b-32768" #  gpt-4-turbo-preview or mixtral-8x7b-32768
)

In [37]:
from langchain.chains import LLMChain
from langchain.memory import ConversationBufferMemory, ReadOnlySharedMemory
from langchain.prompts import PromptTemplate
from langchain_community.tools import DuckDuckGoSearchRun

In [60]:
llm = ChatGroq(temperature=0, groq_api_key="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", model_name="mixtral-8x7b-32768")

In [61]:
from langchain.agents import  Tool
template = """This is a conversation between a human and ai agent:

{chat_history}

Write a summary of the conversation for {input}:
"""

prompt = PromptTemplate(input_variables=["input", "chat_history"], template=template)
memory = ConversationBufferMemory(memory_key="chat_history")
readonlymemory = ReadOnlySharedMemory(memory=memory)
summary_chain = LLMChain(
    llm=llm,
    prompt=prompt,
    verbose=True,
    memory=readonlymemory,  # use the read-only memory to prevent the tool from modifying the memory
)

In [62]:
search =  DuckDuckGoSearchRun()

tool_use = [
    Tool(
        name="Search",
        func=search.run,
        description="useful for when you need to answer questions about current events",
    ),
    Tool(
        name="Summary",
        func=summary_chain.run,
        description="useful for when you summarize a conversation. The input to this tool should be a string, representing who will read this summary.",
    )

]

tool_use_1 = [
    Tool(
        name="Summary",
        func=summary_chain.run,
        description="useful for when you summarize a conversation. The input to this tool should be a string, representing who will read this summary.",
    )
]

In [63]:
# Define Agents
email_author = Agent(
    role='Professional Email Author',
    goal='Craft concise and engaging emails',
    backstory='Experienced in writing impactful marketing emails.',
    verbose=True,
    allow_delegation=False,
    llm=llm,
    tools=tool_use
)
marketing_strategist = Agent(
    role='Marketing Strategist',
    goal='Lead the team in creating effective cold emails',
    backstory='A seasoned Chief Marketing Officer with a keen eye for standout marketing content.',
    verbose=True,
    allow_delegation=True,
    llm=llm,
    tools=tool_use_1
)

content_specialist = Agent(
    role='Content Specialist',
    goal='Critique and refine email content',
    backstory='A professional copywriter with a wealth of experience in persuasive writing.',
    verbose=True,
    allow_delegation=False,
    llm=llm,
    tools=tool_use_1
    )


In [64]:
email_task = Task(
    description='''1. Generate two distinct variations of a cold email promoting a video editing solution.
    2. Evaluate the written emails for their effectiveness and engagement.
    3. Scrutinize the emails for grammatical correctness and clarity.
    4. Adjust the emails to align with best practices for cold outreach. Consider the feedback
    provided to the marketing_strategist.
    5. Revise the emails based on all feedback, creating two final versions.''',
    expected_output='Two final versions of the cold email after considering all feedback.',
    agent=marketing_strategist ) # The Marketing Strategist is in charge and can delegate


In [65]:
# Create a Single Crew
email_crew = Crew(
    agents=[email_author, marketing_strategist, content_specialist],
    tasks=[email_task],
    verbose=True,
    process=Process.sequential
)

# Execution Flow
print("Crew: Working on Email Task")
emails_output = email_crew.kickoff()

Crew: Working on Email Task
 [DEBUG]: == Working Agent: Marketing Strategist
 [INFO]: == Starting Task: 1. Generate two distinct variations of a cold email promoting a video editing solution. 
    2. Evaluate the written emails for their effectiveness and engagement.
    3. Scrutinize the emails for grammatical correctness and clarity.
    4. Adjust the emails to align with best practices for cold outreach. Consider the feedback 
    provided to the marketing_strategist.
    5. Revise the emails based on all feedback, creating two final versions.


> Entering new CrewAgentExecutor chain...
I need to generate two variations of a cold email promoting a video editing solution. I will first create a rough draft for the email and then revise it to improve its effectiveness and engagement. I will also ensure that the email is grammatically correct and aligned with best practices for cold outreach.

Action: Delegate work to co-worker
Action Input: {"coworker": "Content Specialist", "task": "C

In [66]:
# Define Agents
pitch_author = Agent(
    role='Professional Startup Pitch Coach',
    goal='Craft concise and engaging pitches for the startups',
    backstory='Experienced in evaluating and presenting high level product pitches for startups',
    verbose=True,
    allow_delegation=False,
    llm=llm,
    tools=tool_use
)
# marketing_strategist = Agent(
#     role='Marketing Strategist',
#     goal='Lead the team in creating effective cold emails',
#     backstory='A seasoned Chief Marketing Officer with a keen eye for standout marketing content.',
#     verbose=True,
#     allow_delegation=True,
#     llm=llm,
#     tools=tool_use_1
# )

# content_specialist = Agent(
#     role='Content Specialist',
#     goal='Critique and refine email content',
#     backstory='A professional copywriter with a wealth of experience in persuasive writing.',
#     verbose=True,
#     allow_delegation=False,
#     llm=llm,
#     tools=tool_use_1
#     )

In [67]:
business_namer = Agent(
    role='Creative Director for Startup hubs ',
    goal='Give creative names and tag lines for the businesses',
    backstory='A very trendy and updated Creative director who has innovative ideas on businesses and their impact because of the names.',
    verbose=True,
    allow_delegation=True,
    llm=llm,
    tools=tool_use
)



In [68]:
Investment_Advisor = Agent(
    role='Investment Advisor for startups and budding businesses',
    goal='Analyse and guide businesses to take investment in different stages',
    backstory='A professional Investment advisor evaluating risks and benefits in a medium and guides businesses to get funding and make it profitable',
    verbose=True,
    allow_delegation=False,
    llm=llm,
    tools=tool_use_1
    )

In [49]:
# markdown_to_pdf_creator = Agent(
#     role='PDF Converter',
#     goal='Convert the Markdown file to a PDF document. story.md is the markdown file name.',
#     backstory='An efficient converter that transforms Markdown files into professionally formatted PDF documents.',
#     verbose=True,
#     llm=llm,
#     tools=[convermarkdowntopdf],
#     allow_delegation=False
# )

In [69]:
# Create tasks for the agents
task_pitch_generator = Task(
    description='Create a captivating pitch for new and budding businesses to showcase the best out of the businesses and pitchers',
    agent=pitch_author,
    expected_output='A structured outline document containing a captivating pitch with a story in the beginning linking the problem focused and the market research, potential clients etc'
)

task_namegen = Task(
    description='Generate trendy and creative business names with respect to the business category',
    agent=business_namer,
    expected_output='A complete set of suitable names and tag lines for the business based on the trend and market'
)

# task_image_generate = Task(
#     description='Generate 5 images that captures the essence of the children\'s storybook about Animals, aligning with the themes, characters, and narrative outlined for the chapters. Do it one by one.',
#     agent=image_generator,
#     expected_output='A digital image file that visually represents the overarching theme of the children\'s storybook, incorporating elements from the characters and plot as described in the outline. The image should be suitable for inclusion in the storybook as an illustration.',
# )

task_format_content = Task(
    description='Format the pitch  content in markdown covering problem statement , user stories , market research etc',
    agent=pitch_author,
    expected_output='The pitch content formatted in markdown, with each section title followed by the corresponding content',
    context=[task_pitch_generator],
    output_file="story.md"
)

# task_markdown_to_pdf = Task(
#     description='Convert a Markdown file to a PDF document, ensuring the preservation of formatting, structure using the mdpdf library.',
#     agent=markdown_to_pdf_creator,
#     expected_output='A PDF file generated from the Markdown input, accurately reflecting the content with proper formatting. The PDF should be ready for sharing or printing.'
# )



In [52]:
# crew = Crew(
#   agents=[pitch_author, Investment_Advisor, markdown_to_pdf_creator],
#   tasks=[task_pitch_generator,task_namegen, task_format_content,task_markdown_to_pdf],
#   verbose=True,
#   process=Process.sequential
# )

# result = crew.kickoff()

# print(result)

 [DEBUG]: == Working Agent: Professional Startup Pitch Coach
 [INFO]: == Starting Task: Create a captivating pitch for new and budding businesses to showcase the best out of the businesses and pitchers


> Entering new CrewAgentExecutor chain...


NotFoundError: Error code: 404 - {'error': {'message': 'Unknown request URL: POST /openai/v1/openai/v1/chat/completions. Please check the URL for typos, or see the docs at https://console.groq.com/docs/', 'type': 'invalid_request_error', 'code': 'unknown_url'}}

In [70]:
# Create a Single Crew
email_crew = Crew(
    agents=[pitch_author, Investment_Advisor],
    tasks=[task_pitch_generator,task_namegen, task_format_content],
    verbose=True,
    process=Process.sequential
)


In [71]:
result = email_crew.kickoff()

print(result)

 [DEBUG]: == Working Agent: Professional Startup Pitch Coach
 [INFO]: == Starting Task: Create a captivating pitch for new and budding businesses to showcase the best out of the businesses and pitchers


> Entering new CrewAgentExecutor chain...
I need to create a captivating pitch that tells a story and highlights the problem, market research, and potential clients. I will use the Search and Summary tools to gather necessary information.

Action: Search
Action Input: {"query": "importance of storytelling in startup pitches"}
 

Here's why storytelling is an absolute game-changer when it comes to startup pitches: 1. Connecting Emotionally: Stories have the badass superpower of forging deep emotional connections. When you ... Start by understanding why storytelling is a potent tool in the world of investor pitch decks. 2. The Structure of a Compelling Story. Crafting an impactful story involves a strategic approach. Explore the fundamental elements of storytelling, such as a captivating

In [42]:
!pip install -U duckduckgo-search

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.7/5.7 MB 21.2 MB/s eta 0:00:00
